Assume you're given a table Twitter tweet data, write a query to obtain a histogram of tweets posted per user in 2022. Output the tweet count per user as the bucket and the number of Twitter users who fall into that bucket.

In [ ]:
SELECT tweet_count_per_user AS tweet_bucket, COUNT(user_id) users_num
FROM
  (SELECT user_id, COUNT(*) tweet_count_per_user
  FROM tweets
  WHERE to_char(tweet_date::DATE, 'YYYY') = '2022'
  GROUP BY user_id) tweet_per_user
GROUP BY 1


SELECT 
  tweet_count_per_user AS tweet_bucket, 
  COUNT(user_id) AS users_num 
FROM (
  SELECT 
    user_id, 
    COUNT(tweet_id) AS tweet_count_per_user 
  FROM tweets 
  WHERE tweet_date BETWEEN '2022-01-01' 
    AND '2022-12-31'
  GROUP BY user_id) AS total_tweets 
GROUP BY tweet_count_per_user

Given a table of candidates and their skills, you're tasked with finding the candidates best suited for an open Data Science job. You want to find candidates who are proficient in Python, Tableau, and PostgreSQL.

Write a query to list the candidates who possess all of the required skills for the job. Sort the output by candidate ID in ascending order.

Assumption:

There are no duplicates in the candidates table:

| Column Name  | Type    |
| :----------- | :------ |
| candidate_id | integer |
| skill        | varchar |

Example Input:

| candidate_id | skill      |
| :----------- | :--------- |
| 123          | Python     |
| 123          | Tableau    |
| 123          | PostgreSQL |
| 234          | R          |
| 234          | PowerBI    |
| 234          | SQL Server |
| 345          | Python     |
| 345          | Tableau    |


In [ ]:
SELECT candidate_id
FROM candidates
GROUP BY candidate_id
HAVING 'Python' = ANY(ARRAY_AGG(skill))
AND 'Tableau' = ANY(ARRAY_AGG(skill))
AND 'PostgreSQL' = ANY(ARRAY_AGG(skill))
ORDER BY 1

SELECT candidate_id
FROM candidates
WHERE skill IN ('Python', 'Tableau', 'PostgreSQL')
GROUP BY candidate_id
HAVING COUNT(*) = 3
ORDER BY candidate_id

Write a solution to find the second highest salary from the Employee table. If there is no second highest salary, return null (return None in Pandas).

In [ ]:
WITH rankings AS (
    SELECT salary, DENSE_RANK() OVER (ORDER BY salary DESC) rank_
    FROM Employee
)
SELECT COALESCE(
    (SELECT DISTINCT salary FROM rankings WHERE rank_ = 2), NULL) AS "SecondHighestSalary";


SELECT COALESCE(
    (
    SELECT DISTINCT salary
    FROM Employee
    ORDER BY salary DESC
    OFFSET 1 LIMIT 1),
    NULL
) AS "SecondHighestSalary"
FROM Employee
LIMIT 1


SELECT MAX(salary) AS "SecondHighestSalary"
FROM Employee
WHERE salary < (SELECT MAX(salary) FROM Employee)

In [ ]:
import pandas as pd

def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    
    salaries = employee['salary'].drop_duplicates()
    salaries.sort_values(ascending=False, inplace=True)
    return pd.DataFrame({'SecondHighestSalary' : [salaries.iloc[1] if salaries.shape[0] > 1 else None]})